In [49]:
import ast
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import KFold
import joblib

In [50]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

# Para cada amostra, identificar o ansatz com maior acurácia
y_best_ansatz = np.argmax(y, axis=1)  # Retorna índices 0-29

In [51]:
print(y_best_ansatz)

[ 6  0  6  0 11 14  0 14 14  0  0  2  0  6  6 14  2  6  0 11  2 14 14 16
  2  6  6  2 11  6 14  6 16 14 14 14 14  8 14  6  6 14  6 14  6  8 14  6
  6  6 14  6  6  6  6  6  8  6 14  0  6  6  6  6  6  1  6  6  6  2  6  6
  6  6  6  6  6  2  2  6  6  6  6  6  6  6  2  6  2  6 13  0 12  0  7  7
  7  8  0  7  7 12  7  8 21 12 13  7  7  7 12 17  7 13 21  7  7  7  6  7
  6 14  6 25  6  6  1 14  1  6  6  6  6 14 16  6  6  6  6  6  6  6 14  1
  6  6  6 14  6 14  6  6 14 14 14  6 14 14  6  6  6  6 14  6  6  6  6  6
 14  6  6  6  6 14  6  6  6  6  6  6 19 20  3 22 11 19  1  6 20 29 20  0
  0 19 28 29  6 11  6 27  7 17 20 26  0 22 17  3  0 29 13  2 27  0  7 10
 24 24 20  6 10 22 24 22 10 10 14  8 15 17 10 22  6  0 11  2 12 22 27 20
  2  0 15  0 29 10 12  2 17  6  3 29 16  0 11  0  0  7  7  0  6 10 14  0
  6  6  0  3 10  0  2 11 16 18  2 18 29  6 13 16 16 16 22  0 16 23  6 16
 20 18 18 16 16  0 11 16 16 17 16 20  6  8  3 11 11  6 22  3 22  0 29 11
  0 20  2  6 22 28 29  0 12 27 11 26 29 27 28  0 12

In [52]:
# criando kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

#realizando validação cruzada

dt_classifier = DecisionTreeClassifier(random_state=45)

score = cross_val_score(dt_classifier, X, y_best_ansatz, cv=kf)

print("Scores:", score)
print("Média:", score.mean())
print("Desvio padrão:", score.std())

Scores: [0.23333333 0.30833333 0.24166667]
Média: 0.2611111111111111
Desvio padrão: 0.033564016593318265


In [ ]:
# treinando o dt classifier e obtendo os top3 ansatz mais classificados
dt_classifier.fit(X, y_best_ansatz)

# Obter probabilidades de cada ansatz dado o input 6
probabilidades = dt_classifier.predict_proba(X)[6]

# Rankear os top-3 ansatz com maiores probabilidades
top3_indices = np.argsort(probabilidades)[-3:][::-1]  # Ordem decrescente
top3_ansatz = [(idx, probabilidades[idx]) for idx in top3_indices]

print("Top-3 Ansatz Previstos:")
for idx, prob in top3_ansatz:
    print(f"Ansatz {idx} (Probabilidade: {prob:.2f})")


Top-3 Ansatz Previstos:
Ansatz 0 (Probabilidade: 1.00)
Ansatz 27 (Probabilidade: 0.00)
Ansatz 25 (Probabilidade: 0.00)


['./dt_classifier.joblib']

In [ ]:
#salvando o modelo treinado
joblib.dump(dt_classifier, './models_salvos/dt_classifier.joblib')